<a href="https://colab.research.google.com/github/Hewish8/DeepLearningChatbot/blob/main/DeepLearningChatboy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.youtube.com/watch?v=ON5pGUJDNow&list=PLzMcBGfZo4-ndH9FoC4YWHGXG5RZekt-Q&index=2

In [ ]:
pip install nltk

In [ ]:
pip install numpy

In [ ]:
pip install tflearn

In [ ]:
#pip install tensorflow

In [ ]:
pip install tensorflow==1.15

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [ ]:
import numpy
import tensorflow
import tflearn
import random
import json


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks'

with open("intents.json") as file:
  data = json.load(file)

#print(data)
print(data["intents"])

/content/drive/My Drive/Colab Notebooks
[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip c

In [ ]:
words =[]
labels = []
docs = []
docs_x = []
docs_y = []


for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern) #Split all the words
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])

  if intent["tag"] not in labels:
    labels.append(intent["tag"])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [ 0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag = []

  wrds = [stemmer.stem(w.lower()) for w in doc]

  for w in words:
     if w in wrds:
       bag.append(1)
     else:
       bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)



In [ ]:
# Training AI part
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation= "softmax") #No of output is No of tags
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch= 1000, batch_size=8, show_metric=True)
model.save("model.tflearn") #saving the model name as model.tflearn

In [ ]:
#load the model
model.load("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/model.tflearn


In [ ]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  
  return numpy.array(bag)

In [ ]:
def chat():
  print("Hi, I am Kal-ex 2.0. How may I help you? Type quit when you are done")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp, words)])
    #print(results)
    results_index = numpy.argmax(results)
    tag = labels[results_index]
    #print(tag)

#No need to reopen as already loaded
#    with open("intents.json") as file:
#      data = json.load(file)

    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
    print(random.choice(responses))



chat()
